# Flu Shot Learning: Predict H1N1 and Seasonal Flu Vaccines

## Business Understanding
### Overview
In 2009, during the H1N1 influenza outbreak, the U.S. National H1N1 Flu Survey collected extensive data on individuals' vaccination statuses, backgrounds, opinions, and health behaviors. This dataset provides an opportunity to analyze the factors that influenced people's decisions to receive the H1N1 and seasonal flu vaccines.

By predicting vaccination uptake based on these factors, insights can be gained to improve the design and communication strategies for future vaccination campaigns. Public health officials can use these insights to tailor outreach efforts, allocate resources efficiently, and address vaccine hesitancy more effectively.

### Business Problem
The primary goal of this project is to predict whether an individual received the H1N1 flu vaccine based on their demographic information, personal beliefs, and health behaviors. This binary classification task can help public health agencies:

1. Identify patterns among populations who are more or less likely to get vaccinated.
2. Understand barriers to vaccine adoption, such as misconceptions, trust issues, or socio-economic challenges.
3. Develop targeted interventions to increase vaccination rates, especially in communities where uptake is low.
4. Optimize communication strategies by identifying which beliefs and behaviors are most strongly associated with vaccination decisions.

By solving this problem, public health organizations can improve vaccination outreach and preparedness for future epidemics or pandemics, ultimately protecting more people from preventable diseases.

## Data Understanding
The datasets being used for this project was obtained from [Driven Data](https://www.drivendata.org/competitions/66/flu-shot-learning/). Here, I am going to review the dataset to assess the structure and characteristics of the data.

In [1]:
# Import necessary packages

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, roc_curve

In [2]:
# Loading the datasets

X_train = pd.read_csv('Data/training_set_features.csv', index_col='respondent_id')
X_test = pd.read_csv('Data/test_set_features.csv', index_col='respondent_id')
y_train = pd.read_csv('Data/training_set_labels.csv', index_col='respondent_id')

### a) .head()
Displays the first five rows of the data.

In [3]:
X_train.head()

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,...,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation
respondent_id,,,,,,,,,,,,,,,,,,,,,
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,Below Poverty,Not Married,Own,Not in Labor Force,oxchjgsf,Non-MSA,0.0,0.0,NaN,NaN
1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,...,Below Poverty,Not Married,Rent,Employed,bhuqouqj,"MSA, Not Principle City",0.0,0.0,pxcmvdjn,xgwztkwe
2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,...,"<= $75,000, Above Poverty",Not Married,Own,Employed,qufhixun,"MSA, Not Principle City",2.0,0.0,rucpziij,xtkaffoo
3,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,Below Poverty,Not Married,Rent,Not in Labor Force,lrircsnp,"MSA, Principle City",0.0,0.0,NaN,NaN
4,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,...,"<= $75,000, Above Poverty",Married,Own,Employed,qufhixun,"MSA, Not Principle City",1.0,0.0,wxleyezf,emcorrxb


In [4]:
X_test.head()

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,...,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation
respondent_id,,,,,,,,,,,,,,,,,,,,,
26707,2.0,2.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,...,"> $75,000",Not Married,Rent,Employed,mlyzmhmf,"MSA, Not Principle City",1.0,0.0,atmlpfrs,hfxkjkmi
26708,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Below Poverty,Not Married,Rent,Employed,bhuqouqj,Non-MSA,3.0,0.0,atmlpfrs,xqwwgdyp
26709,2.0,2.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,...,"> $75,000",Married,Own,Employed,lrircsnp,Non-MSA,1.0,0.0,nduyfdeo,pvmttkik
26710,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,"<= $75,000, Above Poverty",Married,Own,Not in Labor Force,lrircsnp,"MSA, Not Principle City",1.0,0.0,NaN,NaN
26711,3.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,...,"<= $75,000, Above Poverty",Not Married,Own,Employed,lzgpxyit,Non-MSA,0.0,1.0,fcxhlnwr,mxkfnird


In [5]:
y_train.head()

,h1n1_vaccine,seasonal_vaccine
respondent_id,,
0,0,0
1,0,1
2,0,0
3,0,1
4,0,0


### b) .info()
Gives general information on the data and each column.

In [6]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26707 entries, 0 to 26706
Data columns (total 35 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   h1n1_concern                 26615 non-null  float64
 1   h1n1_knowledge               26591 non-null  float64
 2   behavioral_antiviral_meds    26636 non-null  float64
 3   behavioral_avoidance         26499 non-null  float64
 4   behavioral_face_mask         26688 non-null  float64
 5   behavioral_wash_hands        26665 non-null  float64
 6   behavioral_large_gatherings  26620 non-null  float64
 7   behavioral_outside_home      26625 non-null  float64
 8   behavioral_touch_face        26579 non-null  float64
 9   doctor_recc_h1n1             24547 non-null  float64
 10  doctor_recc_seasonal         24547 non-null  float64
 11  chronic_med_condition        25736 non-null  float64
 12  child_under_6_months         25887 non-null  float64
 13  health_worker   

In [7]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26708 entries, 26707 to 53414
Data columns (total 35 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   h1n1_concern                 26623 non-null  float64
 1   h1n1_knowledge               26586 non-null  float64
 2   behavioral_antiviral_meds    26629 non-null  float64
 3   behavioral_avoidance         26495 non-null  float64
 4   behavioral_face_mask         26689 non-null  float64
 5   behavioral_wash_hands        26668 non-null  float64
 6   behavioral_large_gatherings  26636 non-null  float64
 7   behavioral_outside_home      26626 non-null  float64
 8   behavioral_touch_face        26580 non-null  float64
 9   doctor_recc_h1n1             24548 non-null  float64
 10  doctor_recc_seasonal         24548 non-null  float64
 11  chronic_med_condition        25776 non-null  float64
 12  child_under_6_months         25895 non-null  float64
 13  health_worke

In [ ]:
y_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26707 entries, 0 to 26706
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   h1n1_vaccine      26707 non-null  int64
 1   seasonal_vaccine  26707 non-null  int64
dtypes: int64(2)
memory usage: 625.9 KB


### c) .describe()
Gives summary statistics such as mean, count, etc of columns with numerical data.

In [9]:
X_train.describe()

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,...,health_worker,health_insurance,opinion_h1n1_vacc_effective,opinion_h1n1_risk,opinion_h1n1_sick_from_vacc,opinion_seas_vacc_effective,opinion_seas_risk,opinion_seas_sick_from_vacc,household_adults,household_children
count,26615.000000,26591.000000,26636.000000,26499.000000,26688.000000,26665.000000,26620.00000,26625.000000,26579.000000,24547.000000,...,25903.000000,14433.00000,26316.000000,26319.000000,26312.000000,26245.000000,26193.000000,26170.000000,26458.000000,26458.000000
mean,1.618486,1.262532,0.048844,0.725612,0.068982,0.825614,0.35864,0.337315,0.677264,0.220312,...,0.111918,0.87972,3.850623,2.342566,2.357670,4.025986,2.719162,2.118112,0.886499,0.534583
std,0.910311,0.618149,0.215545,0.446214,0.253429,0.379448,0.47961,0.472802,0.467531,0.414466,...,0.315271,0.32530,1.007436,1.285539,1.362766,1.086565,1.385055,1.332950,0.753422,0.928173
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000
25%,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.00000,0.000000,0.000000,0.000000,...,0.000000,1.00000,3.000000,1.000000,1.000000,4.000000,2.000000,1.000000,0.000000,0.000000
50%,2.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.00000,0.000000,1.000000,0.000000,...,0.000000,1.00000,4.000000,2.000000,2.000000,4.000000,2.000000,2.000000,1.000000,0.000000
75%,2.000000,2.000000,0.000000,1.000000,0.000000,1.000000,1.00000,1.000000,1.000000,0.000000,...,0.000000,1.00000,5.000000,4.000000,4.000000,5.000000,4.000000,4.000000,1.000000,1.000000
max,3.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,...,1.000000,1.00000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,3.000000,3.000000


In [10]:
X_test.describe()

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,...,health_worker,health_insurance,opinion_h1n1_vacc_effective,opinion_h1n1_risk,opinion_h1n1_sick_from_vacc,opinion_seas_vacc_effective,opinion_seas_risk,opinion_seas_sick_from_vacc,household_adults,household_children
count,26623.000000,26586.000000,26629.000000,26495.000000,26689.000000,26668.000000,26636.000000,26626.000000,26580.000000,24548.000000,...,25919.000000,14480.000000,26310.000000,26328.000000,26333.000000,26256.000000,26209.000000,26187.000000,26483.000000,26483.000000
mean,1.623145,1.266042,0.049645,0.729798,0.069279,0.826084,0.351517,0.337227,0.683747,0.222666,...,0.111501,0.887914,3.844622,2.326838,2.360612,4.024832,2.708688,2.143392,0.894310,0.543745
std,0.902755,0.615617,0.217215,0.444072,0.253934,0.379045,0.477453,0.472772,0.465022,0.416044,...,0.314758,0.315483,1.007570,1.275636,1.359413,1.083204,1.376045,1.339102,0.754244,0.935057
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000
25%,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,1.000000,3.000000,1.000000,1.000000,4.000000,2.000000,1.000000,0.000000,0.000000
50%,2.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,...,0.000000,1.000000,4.000000,2.000000,2.000000,4.000000,2.000000,2.000000,1.000000,0.000000
75%,2.000000,2.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,...,0.000000,1.000000,5.000000,4.000000,4.000000,5.000000,4.000000,4.000000,1.000000,1.000000
max,3.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,3.000000,3.000000


In [11]:
y_train.describe()

,h1n1_vaccine,seasonal_vaccine
count,26707.000000,26707.000000
mean,0.212454,0.465608
std,0.409052,0.498825
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,0.000000,1.000000
max,1.000000,1.000000


## Data Preparation

### Data Merging

In [12]:
# Merge x_train and y_train

train_data = pd.merge(X_train, y_train, how='left', on='respondent_id')

### Data Cleaning
This will involve checking for duplicates and missing values and if duplicates or missing values are present in the data, action will be taken appropriately.

#### 1. train_data

In [13]:
# Check for duplicates

train_data.duplicated().sum()

0

There are no duplicated rows in the data.

In [14]:
# Check for missing values

train_data.isna().sum()[train_data.isna().sum() > 0]

h1n1_concern                      92
h1n1_knowledge                   116
behavioral_antiviral_meds         71
behavioral_avoidance             208
behavioral_face_mask              19
behavioral_wash_hands             42
behavioral_large_gatherings       87
behavioral_outside_home           82
behavioral_touch_face            128
doctor_recc_h1n1                2160
doctor_recc_seasonal            2160
chronic_med_condition            971
child_under_6_months             820
health_worker                    804
health_insurance               12274
opinion_h1n1_vacc_effective      391
opinion_h1n1_risk                388
opinion_h1n1_sick_from_vacc      395
opinion_seas_vacc_effective      462
opinion_seas_risk                514
opinion_seas_sick_from_vacc      537
education                       1407
income_poverty                  4423
marital_status                  1408
rent_or_own                     2042
employment_status               1463
household_adults                 249
h

There are missing values.

**Dealing with missing values**

In [15]:
# Function to drop columns with more than 30% missing values

def drop_high_missing_cols(df, threshold=0.3):
    """
    Drops columns with more than a given threshold of missing values.

    Parameters:
    - df (pd.DataFrame): The input DataFrame.
    - threshold (float): The proportion of missing values above which columns are dropped (default is 0.3).

    Returns:
    - pd.DataFrame: The DataFrame with high-missing-value columns dropped.
    """
    # Calculate the proportion of missing values for each column
    missing_proportion = df.isna().mean()
    
    # Identify columns where the missing proportion exceeds the threshold
    cols_to_drop = missing_proportion[missing_proportion > threshold].index
    
    # Drop the identified columns
    df_cleaned = df.drop(columns=cols_to_drop)
    
    return df_cleaned

In [16]:
# Function to fill missing values 

def fill_missing_values(df):
    """
    Fills missing values in the DataFrame:
    - For float columns, fill with the median.
    - For object (string) columns, fill with 'unknown'.

    Parameters:
    - df (pd.DataFrame): The input DataFrame.

    Returns:
    - pd.DataFrame: The DataFrame with missing values filled.
    """
    df_filled = df.copy()

    # Fill float columns with the median
    float_cols = df_filled.select_dtypes(include=['float']).columns
    for col in float_cols:
        median_value = df_filled[col].median()
        df_filled[col].fillna(median_value, inplace=True)

    # Fill object (string) columns with 'unknown'
    object_cols = df_filled.select_dtypes(include=['object']).columns
    for col in object_cols:
        df_filled[col].fillna('unknown', inplace=True)

    return df_filled


In [17]:
# Drop columns with more than 30% of null values
train_data_cleaned = drop_high_missing_cols(train_data, threshold=0.3)

# Fill missing values
train_data_cleaned = fill_missing_values(train_data_cleaned)

In [18]:
# Check if there is still any missing values

train_data_cleaned.isna().sum().sum()

0

There are no longer any missing values.

#### 2. X_test

In [19]:
# Check for duplicates

X_test.duplicated().sum()

0

There are no duplicated rows.

In [20]:
# Check for missing values

X_test.isna().sum()[X_test.isna().sum() > 0]

h1n1_concern                      85
h1n1_knowledge                   122
behavioral_antiviral_meds         79
behavioral_avoidance             213
behavioral_face_mask              19
behavioral_wash_hands             40
behavioral_large_gatherings       72
behavioral_outside_home           82
behavioral_touch_face            128
doctor_recc_h1n1                2160
doctor_recc_seasonal            2160
chronic_med_condition            932
child_under_6_months             813
health_worker                    789
health_insurance               12228
opinion_h1n1_vacc_effective      398
opinion_h1n1_risk                380
opinion_h1n1_sick_from_vacc      375
opinion_seas_vacc_effective      452
opinion_seas_risk                499
opinion_seas_sick_from_vacc      521
education                       1407
income_poverty                  4497
marital_status                  1442
rent_or_own                     2036
employment_status               1471
household_adults                 225
h

There are missing values.

**Dealing with missing values**

In [21]:
# Drop columns with more than 30% of null values
X_test_cleaned = drop_high_missing_cols(X_test, threshold=0.3)

# Fill missing values
X_test_cleaned = fill_missing_values(X_test_cleaned)

In [22]:
# Check if there is still any missing values

X_test_cleaned.isna().sum().sum()

0

There are no longer any missing values.

### Encoding
Converting categorical variables into numerical values. 

In [23]:
# Function to encode categorical columns

def encode_categorical_columns(df):
    """
    Encodes categorical columns in a DataFrame.
    - One-hot encodes nominal categorical columns (with >2 unique values).
    - Label-encodes binary categorical columns (with 2 unique values).

    Parameters:
        df (pd.DataFrame): The DataFrame containing categorical columns.

    Returns:
        pd.DataFrame: The DataFrame with encoded categorical columns.
    """
    # Create a copy to avoid modifying the original DataFrame
    df_encoded = df.copy()
    
    # Identify categorical columns
    categorical_cols = df_encoded.select_dtypes(include=['object']).columns

    # Initialize LabelEncoder for binary encoding
    label_encoder = LabelEncoder()

    for col in categorical_cols:
        unique_vals = df_encoded[col].dropna().unique()
        
        # If the column has exactly 2 unique values, apply Label Encoding (binary)
        if len(unique_vals) == 2:
            df_encoded[col] = label_encoder.fit_transform(df_encoded[col])
        # If the column has more than 2 unique values, apply One-Hot Encoding (nominal)
        elif len(unique_vals) > 2:
            df_encoded = pd.get_dummies(df_encoded, columns=[col], drop_first=True)

    return df_encoded


In [24]:
# Encoding train_data_cleaned
train_data_encoded = encode_categorical_columns(train_data_cleaned)

# Encoding X_test_cleaned
X_test_encoded = encode_categorical_columns(X_test_cleaned)

## Modelling

I will be using the **Random Forest Classifier** model.

In [25]:
# Define features and target variables
X = train_data_encoded.drop(columns=['h1n1_vaccine', 'seasonal_vaccine'])
y_h1n1 = train_data_encoded['h1n1_vaccine']
y_seasonal = train_data_encoded['seasonal_vaccine']

# Split the data into training and validation sets
X_train, X_val, y_h1n1_train, y_h1n1_val, y_seasonal_train, y_seasonal_val = train_test_split(
    X, y_h1n1, y_seasonal, test_size=0.2, random_state=42
)

# Initialize Random Forest models
h1n1_rf = RandomForestClassifier(random_state=42, class_weight='balanced')
seasonal_rf = RandomForestClassifier(random_state=42)

# Train the models
h1n1_rf.fit(X_train, y_h1n1_train)
seasonal_rf.fit(X_train, y_seasonal_train)

# Predict probabilities on the validation set
h1n1_val_preds = h1n1_rf.predict_proba(X_val)[:, 1]
seasonal_val_preds = seasonal_rf.predict_proba(X_val)[:, 1]

**Submission**

In [26]:
# Predict probabilities for the test set
y_pred_proba_h1n1 = h1n1_rf.predict_proba(X_test_encoded)[:, 1]
y_pred_proba_seasonal = seasonal_rf.predict_proba(X_test_encoded)[:, 1]

# Format prediction probabilities for submission
submission = pd.DataFrame({
    'h1n1_vaccine': y_pred_proba_h1n1,
    'seasonal_vaccine': y_pred_proba_seasonal
})

# Set and name the index
submission.index.name = 'respondent_id'

# Save to CSV
submission.to_csv('Data/submission.csv')

## Evaluation

The model's performance will be evaluated by use of roc-auc score.

In [27]:
# Calculate ROC AUC scores
h1n1_roc_auc = roc_auc_score(y_h1n1_val, h1n1_val_preds)
seasonal_roc_auc = roc_auc_score(y_seasonal_val, seasonal_val_preds)

# Calculate the mean ROC AUC score
overall_roc_auc = (h1n1_roc_auc + seasonal_roc_auc) / 2

print(f"H1N1 Vaccine ROC AUC: {h1n1_roc_auc:.5f}")
print(f"Seasonal Vaccine ROC AUC: {seasonal_roc_auc:.5f}")
print(f"Overall ROC AUC Score: {overall_roc_auc:.5f}")

H1N1 Vaccine ROC AUC: 0.82392
Seasonal Vaccine ROC AUC: 0.84755
Overall ROC AUC Score: 0.83573


- **H1N1 Vaccine ROC AUC: 0.8239**: This indicates good predictive performance for the H1N1 vaccine, as values above 0.8 suggest a strong model.

- **Seasonal Vaccine ROC AUC: 0.8475**: This shows even better performance for predicting the seasonal flu vaccine.

✅ **Overall Score: 0.8357**

This is a strong performance, suggesting the model is well-calibrated and performing consistently across both target variables.